In [ ]:
!pip install tensorflow

import tensorflow as tf
import numpy as np
import os

# Загрузка текстовых данных
file_path = '/content/mertvye-dushi.txt'  # Убедитесь, что файл загружен
text = open(file_path, 'r', encoding='utf-8').read()

# Создание словаря символов
vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}  # Символ -> индекс
idx2char = np.array(vocab)  # Индекс -> символ

# Преобразование текста в числовые последовательности
text_as_int = np.array([char2idx[c] for c in text])

# Создание последовательностей
seq_length = 100  # Длина последовательности
examples_per_epoch = len(text) // (seq_length + 1)

# Создание tf.data.Dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Разделение на последовательности
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

# Функция для разделения на входные и целевые данные
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# Применение функции к данным
dataset = sequences.map(split_input_target)

# Подготовка данных для обучения
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Параметры модели
vocab_size = len(vocab)  # Размер словаря
embedding_dim = 256  # Размерность embedding-слоя
rnn_units = 1024  # Количество нейронов в GRU

# Создание модели
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=None),  # Убрали batch_input_shape
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

# Создание модели
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

# Компиляция модели
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Обучение модели
EPOCHS = 100  # Количество эпох
history = model.fit(dataset, epochs=EPOCHS)

# Сохранение модели
model.save('text_generation_model.h5')

Epoch 1/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step - loss: 3.4807
Epoch 2/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 16s 52ms/step - loss: 2.2755
Epoch 3/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - loss: 2.0231
Epoch 4/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 8s 53ms/step - loss: 1.8303
Epoch 5/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - loss: 1.6910
Epoch 6/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - loss: 1.5951
Epoch 7/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 8s 53ms/step - loss: 1.5279
Epoch 8/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - loss: 1.4633
Epoch 9/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - loss: 1.4149
Epoch 10/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step - loss: 1.3652
Epoch 11/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - loss: 1.3218
Epoch 12/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - loss: 1.2800
Epoch 13/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - loss: 1.2380
Epoch 14/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - loss: 1.1954
Epoch 15/100
117/117 ━━